In [1]:
import numpy as np
import cv2
import pickle 
from cv2 import VideoCapture
from cv2 import waitKey
from keras.models import load_model

In [2]:
framewidth = 640
frameHeight = 480
brightness = 180
threshold = 0.95
font = cv2.FONT_HERSHEY_SIMPLEX

In [3]:
#setup video camera
cap = cv2.VideoCapture(0) #0은 카메라 번호(노트북 내부 웹캠)
cap.set(3, framewidth)
cap.set(4, frameHeight)
cap.set(10, brightness)
model = load_model("C:/Users/k2h2j3/trafficdata/trafficsign/my_model2.h5")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        25632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 64)          36928     
                                                        

In [4]:
def grayscale(img):
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #색깔 bgr을 회색조로 변환
    return img

def equalize(img):
    img = cv2.equalizeHist(img) #균일화
    return img

def preprocessing(img): #영상 이미지 전처리
    img = grayscale(img)
    img = equalize(img)
    img = img/255
    return img

def getCalssName(classIndex): 
    if classIndex == 0: 
        return 'speed Limit 30km/h'
    elif classIndex == 1: 
        return 'speed Limit 50km/h'
    elif classIndex == 2: 
        return 'speed Limit 100km/h'
    elif classIndex == 3: 
        return 'Right-of-way at the next intersection'
    elif classIndex == 4: 
        return 'Stop'
    elif classIndex == 5: 
        return 'Veh > 3.5 tons prohibited'
    elif classIndex == 6: 
        return 'Road work'
    elif classIndex == 7: 
        return 'Turn right ahead'
    elif classIndex == 8: 
        return 'Turn left ahead'
    elif classIndex == 9: 
        return 'Ahead only'
    

In [ ]:
while True:
 
# READ IMAGE
    success, imgOrignal = cap.read()

    img = np.asarray(imgOrignal)
    img = cv2.resize(img, (32, 32))
    img = preprocessing(img)
    cv2.imshow("Processed Image", img)
    img = img.reshape(1, 32, 32, 1)
    cv2.putText(imgOrignal, "CLASS: " , (20, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(imgOrignal, "PROBABILITY: ", (20, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
    predictions = model.predict(img)
    classIndex = np.argmax(predictions[0])
    probabilityValue =np.amax(predictions)
   
    if probabilityValue > threshold:
        cv2.putText(imgOrignal,str(classIndex)+" "+str(getCalssName(classIndex)), (120, 35), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA) 
        cv2.putText(imgOrignal, str(round(probabilityValue*100,2) )+"%", (180, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)        
    cv2.imshow("Result", imgOrignal)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break